# This file scraps spotify data from spotify top 200 charts daily 
## The range of date to scrap the data from is the input to the main function

In [1]:
import datetime
import os
from os import system
from datetime import timedelta, date
import pandas as pd


In [2]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [3]:
def download_raw_data(start_date, end_date):
    print("Getting CSVs")
    for single_date in daterange(start_date, end_date):
        #print(single_date.strftime("Processing %Y-%m-%d"))
        
        file_name = single_date.strftime("%Y-%m-%d.csv")
        #date_scrape=pd.read_csv("%Y-%m-%d.csv",skiprows=1)
        spotify_url = single_date.strftime("https://spotifycharts.com/regional/global/daily/%Y-%m-%d/download")
        #download file using wget to the current directory
        #system('wget -c \"{}\"'.format(file_url) + r" --output-document={}".format(os.getcwd()) + "/" + the_date_filename)
        #print('wget -c \"{}\" --output-document={}/{}'.format(file_url, os.getcwd(),the_date_filename))
        system('wget -c \"{}\" --output-document={}/{}'.format(spotify_url, os.getcwd(),file_name))
        print(f"downloading {file_name}...")

In [4]:
def clean_raw_data(start_date, end_date):
    for single_date in daterange(start_date, end_date):
        file_name = "%s.csv" % single_date 
        print(file_name)
        data = pd.read_csv(file_name,skiprows=1)
        #print(data.head())
        urls = data['URL']
        list1 = []
        for url in urls:
            trackID = url.split("/")[-1]
            list1.append(trackID)
            #print(trackID)
        data["trackID"] = list1
        data.to_csv(file_name, index=False)
        print(data.head())

In [5]:
import spotipy

client_id = '90d4bf0d722b4e6892fedd3eb7dea15d'
client_secret = 'b85dd00354944ae4a37e1b1aa7871f5d'

from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [6]:
import csv
import ast
csv_columns = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','type','id','uri','track_href','analysis_url','duration_ms','time_signature']
csv_file = "5-updated-regional-global-daily-latest.csv"

def add_audio_features(start_date, end_date):
    for single_date in daterange(start_date, end_date):
        count = 0
        file_name = single_date.strftime("%Y-%m-%d.csv")
        data = pd.read_csv(file_name)
        trackIDs = data['trackID']
        temp_file = "temp_" + file_name
        output_file = "feature_" + file_name
        try:
            with open(temp_file, 'a') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
                writer.writeheader()

                for trackID in trackIDs:
                    track_features = sp.audio_features(trackID)[0]
                    writer.writerow(track_features)
                    count += 1
                    #print (count, track_features)
                    print(count)
        except IOError:
            print("I/O error")
        reader = csv.reader(open(file_name, 'r'))
        reader1 = csv.reader(open(temp_file, 'r'))
        writer = csv.writer(open(output_file, 'w'))
        for row in reader:
            row1 = next(reader1)
            writer.writerow(row + row1)
        os.remove(temp_file)

In [7]:
def main(start_date, end_date):
    download_raw_data(start_date, end_date)
    clean_raw_data(start_date, end_date)
    add_audio_features(start_date, end_date)

## Change the start_date and end_date to set the range of date of the chart to scrap from

In [ ]:
start_date = date(2020, 10, 1)
end_date = date(2020, 10, 10)
main(start_date, end_date)

Getting CSVs
